In [1]:
# Utiles
    # Lib: https://github.com/sammchardy/python-binance
    # Binance doc : https://github.com/binance-exchange/binance-official-api-docs/blob/master/rest-api.md
    
from binance.client import Client
api_key = '8te7KGDgi6ayc8slY46KbWztB7yuIZ533fCrLWeqARAxBKAuQiMyyfIM4mfDrSlk'
api_secret = 'ykwjZzkYF4Bvh0T7L1OokHfHTnVjKJ4OIvaEXkV97NisgI4JwDW7jPd3pzeVVpHh'

client = Client(api_key, api_secret)

In [2]:
# Others

# For a trading pair status = 'TRADING'
client.get_symbol_info('BNBBTC')

# Get server time
client.get_server_time()

{'serverTime': 1545828298436}

In [2]:
# ============= Check before trading =============

# ping
client.ping()

# Check system status
status = client.get_system_status()
if (status['msg'] == 'normal') and (status['status'] == 0):
    print('ok')
    
# Check account status
status_client = client.get_account_status()
if (status_client['msg'] == 'Normal') and (status_client['success'] == True):
    print('ok')
    
# Check account status - trading allowed
info_client = client.get_account()
if info_client['canTrade'] == True:
    print('ok')

# Exchange info to verify tradingpair exists / trading
exchange_info = client.get_exchange_info()
for symbol in exchange_info['symbols']:
    # si symbol['symbol'] dans trading pairs
    # symbol['status'] doit être égal à TRADING sinon msg erreur + trade pas dessus sur cette itération



{'msg': 'Normal', 'success': True}

In [54]:
for key, value in symbol.items() :
    print (key)

symbol
status
baseAsset
baseAssetPrecision
quoteAsset
quotePrecision
orderTypes
icebergAllowed
filters


In [11]:
# ============== get_price() ==============
    # => get_portfolio_value() => see up, retrieve ticker
    # => create_order() :
        # Side=Buy: get_order_book() + asks (mettre un peu plus haut ?)
        # Side=Sell: get_order_book + bids (mettre un peu plus bas ?)
# get market depth (see what should be approx the price, use to determine limit to put to be sure to buy / sell directly)
depth = client.get_order_book(symbol='ETHUSDT')
depth


{'lastUpdateId': 242253488,
 'bids': [['103.57000000', '1.00000000', []],
  ['103.56000000', '4.69987000', []],
  ['103.55000000', '92.88386000', []],
  ['103.54000000', '9.83711000', []],
  ['103.50000000', '9.18172000', []],
  ['103.47000000', '2.45583000', []],
  ['103.46000000', '5.00000000', []],
  ['103.40000000', '0.36704000', []],
  ['103.35000000', '1.77000000', []],
  ['103.34000000', '0.19408000', []],
  ['103.30000000', '24.89691000', []],
  ['103.29000000', '15.60437000', []],
  ['103.27000000', '46.20000000', []],
  ['103.26000000', '41.24230000', []],
  ['103.25000000', '4.95795000', []],
  ['103.24000000', '0.12695000', []],
  ['103.23000000', '52.00000000', []],
  ['103.20000000', '31.00775000', []],
  ['103.18000000', '116.80557000', []],
  ['103.15000000', '0.09998000', []],
  ['103.12000000', '0.10560000', []],
  ['103.11000000', '1.20000000', []],
  ['103.08000000', '5.90373000', []],
  ['103.07000000', '19.99998000', []],
  ['103.06000000', '0.10675000', []],
  ['

In [64]:
# bid: maximum price buyer willing to pay
# ask: minimum price seller willing to receive
float(depth['bids'][0][0])


True

In [14]:
if 'asks' in depth:
    ask_price = float(depth['asks'][0][0])
    buy_limit_price = ask_price + (ask_price * 0.01)
print(buy_limit_price)

104.6158


In [15]:
float(depth['asks'][0][0])

103.58

In [24]:
# SHOULD NOT BE THE WAY I USE
# get all symbol prices
prices = client.get_all_tickers()

# get one symbol price
prices_dict = { value["symbol"]: value["price"] for value in prices }
float(prices_dict['VETUSDT'])

0.0042

In [25]:
# ============== get_available_amount_crypto()  ==============
# balance for an asset
balance = client.get_asset_balance(asset='VET')
float(balance['free'])

70648.9

In [31]:
# ============== create_order() ==============
def format_amount_order(amount):
    precision = 5
    amt_str = "{:0.0{}f}".format(amount, precision)
    return amt_str

# limit buy
order = client.order_limit_buy(
    symbol='BNBBTC',
    quantity=1000,
    price=format_amount_order(0.00001)) # Use Ask + n%

# limit sell
order = client.order_limit_sell(
    symbol='BNBBTC',
    quantity=1000,
    price=format_amount_order(0.00001)) # Use Bid - n%

In [29]:
order = client.order_limit_sell(
    symbol='VETUSDT',
    quantity=5000,
    price=format_amount_order(0.0055)) # Use Bid - n%

In [40]:
order['orderId']
#{'symbol': 'VETUSDT',
# 'orderId': 9358183,
# 'clientOrderId': 'ILNyxOioi7cPBDGkQSKtbH',
# 'transactTime': 1545235888618,
# 'price': '0.00550000',
# 'origQty': '5000.00000000',
# 'executedQty': '0.00000000',
# 'cummulativeQuoteQty': '0.00000000',
# 'status': 'NEW',
# 'timeInForce': 'GTC',
# 'type': 'LIMIT',
# 'side': 'SELL',
# 'fills': []}

9358183

In [28]:
order = client.order_limit_buy(
    symbol='VETUSDT',
    quantity=5000,
    price=format_amount_order(0.0055)) # Use Ask + n%

BinanceAPIException: APIError(code=-2010): Account has insufficient balance for requested action.

In [36]:
# ============== cancel_open_orders()  ==============
orders = client.get_open_orders(symbol='VETUSDT')
orders
# boucle:
# result = client.cancel_order(
#     symbol='BNBBTC',
#     orderId='orderId')
# Attention, sell order <> buy order ! cancel buy ok, cancel sell ? risqué..

# [{'symbol': 'VETUSDT',
#   'orderId': 9358183,
#   'clientOrderId': 'ILNyxOioi7cPBDGkQSKtbH',
#   'price': '0.00550000',
#   'origQty': '5000.00000000',
#   'executedQty': '0.00000000',
#   'cummulativeQuoteQty': '0.00000000',
#   'status': 'NEW',
#   'timeInForce': 'GTC',
#   'type': 'LIMIT',
#   'side': 'SELL',
#   'stopPrice': '0.00000000',
#   'icebergQty': '0.00000000',
#   'time': 1545235888618,
#   'updateTime': 1545235888618,
#   'isWorking': True}]

[{'symbol': 'VETUSDT',
  'orderId': 9358183,
  'clientOrderId': 'ILNyxOioi7cPBDGkQSKtbH',
  'price': '0.00550000',
  'origQty': '5000.00000000',
  'executedQty': '0.00000000',
  'cummulativeQuoteQty': '0.00000000',
  'status': 'NEW',
  'timeInForce': 'GTC',
  'type': 'LIMIT',
  'side': 'SELL',
  'stopPrice': '0.00000000',
  'icebergQty': '0.00000000',
  'time': 1545235888618,
  'updateTime': 1545235888618,
  'isWorking': True}]

In [37]:
result = client.cancel_order(
     symbol='VETUSDT',
     orderId='9358183')
result

# {'symbol': 'VETUSDT',
#  'origClientOrderId': 'ILNyxOioi7cPBDGkQSKtbH',
#  'orderId': 9358183,
#  'clientOrderId': '4ZPYz4mDfcTg6Ho0O1QhoA',
#  'price': '0.00550000',
#  'origQty': '5000.00000000',
#  'executedQty': '0.00000000',
#  'cummulativeQuoteQty': '0.00000000',
#  'status': 'CANCELED',
#  'timeInForce': 'GTC',
#  'type': 'LIMIT',
#  'side': 'SELL'}

{'symbol': 'VETUSDT',
 'origClientOrderId': 'ILNyxOioi7cPBDGkQSKtbH',
 'orderId': 9358183,
 'clientOrderId': '4ZPYz4mDfcTg6Ho0O1QhoA',
 'price': '0.00550000',
 'origQty': '5000.00000000',
 'executedQty': '0.00000000',
 'cummulativeQuoteQty': '0.00000000',
 'status': 'CANCELED',
 'timeInForce': 'GTC',
 'type': 'LIMIT',
 'side': 'SELL'}

In [48]:
client.get_all_orders(symbol='VETUSDT')

[{'symbol': 'VETUSDT',
  'orderId': 9358183,
  'clientOrderId': 'ILNyxOioi7cPBDGkQSKtbH',
  'price': '0.00550000',
  'origQty': '5000.00000000',
  'executedQty': '0.00000000',
  'cummulativeQuoteQty': '0.00000000',
  'status': 'CANCELED',
  'timeInForce': 'GTC',
  'type': 'LIMIT',
  'side': 'SELL',
  'stopPrice': '0.00000000',
  'icebergQty': '0.00000000',
  'time': 1545235888618,
  'updateTime': 1545236067317,
  'isWorking': True}]

In [38]:
orders = client.get_open_orders(symbol='VETUSDT')
orders
# empty: []

[]

In [49]:
client.get_all_orders(symbol='BNBBTC')

BinanceAPIException: APIError(code=-1121): Invalid symbol.

In [43]:
order = client.get_order(
    symbol='VETUSDT',
    orderId='9358183')
order

# {'symbol': 'VETUSDT',
#  'orderId': 9358183,
#  'clientOrderId': 'ILNyxOioi7cPBDGkQSKtbH',
#  'price': '0.00550000',
#  'origQty': '5000.00000000',
#  'executedQty': '0.00000000',
#  'cummulativeQuoteQty': '0.00000000',
#  'status': 'CANCELED',
#  'timeInForce': 'GTC',
#  'type': 'LIMIT',
#  'side': 'SELL',
#  'stopPrice': '0.00000000',
#  'icebergQty': '0.00000000',
#  'time': 1545235888618,
#  'updateTime': 1545236067317,
#  'isWorking': True}

{'symbol': 'VETUSDT',
 'orderId': 9358183,
 'clientOrderId': 'ILNyxOioi7cPBDGkQSKtbH',
 'price': '0.00550000',
 'origQty': '5000.00000000',
 'executedQty': '0.00000000',
 'cummulativeQuoteQty': '0.00000000',
 'status': 'CANCELED',
 'timeInForce': 'GTC',
 'type': 'LIMIT',
 'side': 'SELL',
 'stopPrice': '0.00000000',
 'icebergQty': '0.00000000',
 'time': 1545235888618,
 'updateTime': 1545236067317,
 'isWorking': True}